## Imports

In [58]:
import pandas as pd
import random
import sys
from docplex.cp.model import CpoModel

In [57]:
# convert a word represented as an array of chars into a string
def convert_array_to_string(w):
    str = ""
    for i in range(len(w)):
        str += w[i]
    return str

# get a set of words as strings 
def get_set_of_words(fileName):
    words_raw = pd.read_csv(fileName)
    words = words_raw['words']
    Words = set()
    for i in range(len(words)):
        if len(words[i]) > 5:
            continue
        Words.add(convert_array_to_string(words[i]))
    return Words

In [56]:
def ct_freq(w):
    ct = [0]*26
    for l in w:
        ct[l]+=1
    return ct

def tuple_ord(Words):   # Return a set of tuples 
    Words=list(Words)
    s=list()
    con_word=list()
    for i in range(len(Words)):
        for j in range(len(Words[i])):
            s.append(ord(Words[i][j])-ord('a'))              
        con_word.append(tuple(s))
        s=list()
    return con_word



In [55]:
def ord_to_str(w):    #Takes list/tuple as input and return a word string as output
    s=""
    for i in range(5):
        s=s+chr(w[i]+ord('a'))
    return s

def ord_to_str_list(Words):   # Return a list of tuples 
    ord_word = []
    for w in Words:
        word_ord = []
        for l in w:
            word_ord.append(ord(l)-ord('a'))
        ord_word.append(tuple(word_ord))   
    return ord_word 

In [64]:
# this function returns a guess that is consistent with the prior history of guesses and the set of Words
# input:
#       a set of words represented as strings
#       a history of prior guesses and their assessments: 
#       the history is an array whose elements are (guess,nb_correct,nb_present) where
#       guess is a string, nb_correct is the number of correct letters and nb_position is the number of letters 
#       that are present but incorrectly placed. 
# output:
#       a new guess as a string

def guess(Words,history):
    if len(history)==0: # samples random word for initial guess
        return random.sample(Words,1)[0]
        
    w_ord = tuple_ord(Words)
    hist_set = set()
    for h in history:
        hist_set.add(h[0])
    ord_hist = tuple_ord(hist_set)
    
    hist_list = []
    for h in history:
        hist_list.append(h[0])
    ord_hist_list = ord_to_str_list(hist_list)
    
    # create model 
    m = CpoModel()
    # dvars
    w = m.integer_var_list(5,min=0,max=25,name="w")
    frequency = m.integer_var_list(26,min=0,max=5,name="frequency")
    
    freq = []
    idx=0
    
    for guess in ord_hist_list:
        freq.append(ct_freq(guess))
    
    # Constraint 0: Table Constraints
    m.add(m.allowed_assignments(w, w_ord))
    m.add(m.forbidden_assignments(w, ord_hist))
    
    # Constraint 1: Correct constraints
    for guess in history:
        prev_guess=guess[0] # String
        m.add(m.sum([(w[i]== (ord(prev_guess[i])-ord('a'))) for i in range(5)])==guess[1]) 
        idx+=1
    
    # Constraint 2: Incorrect Constraint
    for i in range(len(history)):
        m.add(m.sum([m.min(frequency[ord_letter], freq[i][ord_letter]) for ord_letter in range(26)])==history[i][1]+history[i][2]) # No. Incorrect Constraint    
    
    # Constraint 3: Frequency constraint
    for i in range(26):
        m.add(frequency[i]==m.count(w, i))

    msol = m.solve(LogVerbosity='Quiet')
    
    sol=[]
    for i in range(5):
        sol.append(msol[w[i]])
    
    return ord_to_str(sol)

In [53]:
# this function returns a secret word from the set of Words

def select_secret_word(Words):
    return random.sample(Words,1)[0]

In [52]:
# this function computes the score (nb_correct,nb_present) for the guess given the secret word

def compute_scores(secret,guess):
    nbCorrect=0
    nbPresent=0
    correct_letters=""
    present_letters=""
    for i in range(len(secret)):
        if(secret[i]==guess[i]):
            nbCorrect=nbCorrect+1
            correct_letters=correct_letters+secret[i] 
    
    for i in range(len(secret)):
        if((secret[i] in guess) and (secret[i]!=guess[i]) and (secret[i] not in correct_letters)):
            if(secret[i] in present_letters):
                continue
            nbPresent=nbPresent+1
            present_letters=present_letters+secret[i]
    return (nbCorrect,nbPresent)

In [62]:
# this function plays the game with the list of words in the file name
def play_wordle(fileName):
    Words = get_set_of_words(fileName)
    secret = select_secret_word(Words)
    print("secret:",secret)
    history = []
    i = 0
    while True:
        g = guess(Words,history)
        print("guess :",g) 
        (nbCorrect,nbPresent) = compute_scores(secret,g)
        print("\t correct:",nbCorrect," present:",nbPresent)
        if nbCorrect == len(secret):
            break
        history.append((g,nbCorrect,nbPresent))

In [65]:
play_wordle('5-letter-words.csv')

secret: parts
guess : gives
	 correct: 1  present: 0
guess : bowls
	 correct: 1  present: 0
guess : newer
	 correct: 0  present: 1
guess : czars
	 correct: 1  present: 2
guess : yards
	 correct: 3  present: 0
guess : parks
	 correct: 4  present: 0
guess : parts
	 correct: 5  present: 0
